In [ ]:
!pip install scikit-learn
import tensorflow as tf
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.activations import relu, softmax, sigmoid
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

**Mounting Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Function used**


*  To trim dataset:  https://drive.google.com/file/d/1QADrY4kxUQE5P3Nud32PFe5-a51zTZDu/view?usp=drive_link

compile(data_ptr -> pandas dataframe file, fill_mode -> str)

In [ ]:
import pandas as pd
import seaborn as sns
data = pd.read_csv('/content/drive/My Drive/Statistella/train.csv')
# data = data.select_dtypes(exclude = ['object'])
# corr_mat = data.corr()
# sns.heatmap(corr_mat, annot = False, cmap = 'coolwarm')
# plt.title('Correlation Matrix')
# plt.show()

In [ ]:
# target = []
# m = 0
# for i in corr_mat:
#   m+=1
#   target.append(i)
# print(target)
# print(m)
data.drop([
"CODE_GENDER",
"DAYS_REGISTRATION",
"FLAG_EMP_PHONE",
"FLAG_MOBIL",
"FLAG_WORK_PHONE",
"HOUR_APPR_PROCESS_START",
"LIVE_CITY_NOT_WORK_CITY",
"LIVE_REGION_NOT_WORK_REGION",
"NAME_TYPE_SUITE",
"OCCUPATION_TYPE",
"ORGANIZATION_TYPE",
"REG_CITY_NOT_LIVE_CITY",
"REG_CITY_NOT_WORK_CITY",
"REG_REGION_NOT_LIVE_REGION",
"REG_REGION_NOT_WORK_REGION",
"REGION_POPULATION_RELATIVE",
"REGION_RATING_CLIENT",
"WEEKDAY_APPR_PROCESS_START",
"ID"], axis =1)

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from pandas.api.types import is_numeric_dtype

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

for column in data.columns:
    if not is_numeric_dtype(data[column]):
      data[column] = encoder.fit_transform(data[[column]])

import importlib.util, os
spec = importlib.util.spec_from_file_location("fill", "/content/drive/My Drive/Statistella/data_compiler.py")
fill = importlib.util.module_from_spec(spec)
spec.loader.exec_module(fill)

fill.compile(data, "mean");

# print(data.head(20))
from sklearn.model_selection import train_test_split
Y = data['TARGET']
X = data.drop('TARGET', axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:



model1 = Sequential([
    Dense(units = 8, activation = 'relu', bias_regularizer = regularizers.L1(0.01)),
    Dense(units = 4, activation = 'relu'),
    Dense(units = 1, activation = 'sigmoid'),
])
model1.compile(optimizer = Adam(learning_rate = 1e-5), loss = BinaryCrossentropy())
model1.fit(X_train, Y_train, epochs = 30)


Epoch 1/30
5382/5382 [==============================] - 9s 2ms/step - loss: 45715.7695
Epoch 2/30
5382/5382 [==============================] - 8s 2ms/step - loss: 408.5279
Epoch 3/30
5382/5382 [==============================] - 9s 2ms/step - loss: 65.6769
Epoch 4/30
5382/5382 [==============================] - 9s 2ms/step - loss: 12.3112
Epoch 5/30
5382/5382 [==============================] - 8s 2ms/step - loss: 6.9570
Epoch 6/30
5382/5382 [==============================] - 9s 2ms/step - loss: 5.1085
Epoch 7/30
5382/5382 [==============================] - 9s 2ms/step - loss: 4.1514
Epoch 8/30
5382/5382 [==============================] - 9s 2ms/step - loss: 3.5665
Epoch 9/30
5382/5382 [==============================] - 9s 2ms/step - loss: 3.1066
Epoch 10/30
5382/5382 [==============================] - 9s 2ms/step - loss: 2.7494
Epoch 11/30
5382/5382 [==============================] - 8s 2ms/step - loss: 2.5334
Epoch 12/30
5382/5382 [==============================] - 9s 2ms/step - loss: 

In [ ]:
model1.save('/content/drive/My Drive/Statistella/model1.h5')
# Y_pred = model1.predict(X_test)
# submission = []



# # print(Y_pred)

# from sklearn.metrics import roc_curve

# # Compute ROC curve
# fpr, tpr, thresholds = roc_curve(Y_test, Y_pred)

# # Optimal threshold (You can choose based on your criterion)
# optimal_idx = (tpr - fpr).argmax()
# optimal_threshold = thresholds[optimal_idx]
# print("Optimal Threshold:", optimal_threshold)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
Y_pred = model1.predict(X_test)
submission = []
submission = (Y_pred > 0.075).astype(int)
submission = submission.reshape(Y_test.shape)

print(classification_report(Y_test, submission))

1346/1346 [==============================] - 2s 1ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     39629
           1       0.08      1.00      0.15      3423

    accuracy                           0.08     43052
   macro avg       0.04      0.50      0.07     43052
weighted avg       0.01      0.08      0.01     43052



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
Y_pred = model1.predict(X_test)

print(classification_report(Y_test, Y_pred))

1346/1346 [==============================] - 2s 1ms/step


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
drive.flush_and_unmount()